**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Mon Feb 15 13:26:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
#!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_opencv.cpp:1130

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3-w"

mkdir: cannot create directory ‘/mydrive/yolov3-w’: File exists


In [ ]:
!echo "Tiger" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

mkdir: cannot create directory ‘data/obj’: File exists


In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-02-15 13:28:48--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.1’

darknet53.conv.74.1 100%[===================>] 154.96M  43.5MB/s    in 3.9s    

2021-02-15 13:28:53 (39.3 MB/s) - ‘darknet53.conv.74.1’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
replace data/obj/003f38058fbe4fb3.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/obj/003f38058fbe4fb3.jpg  
replace data/obj/003f38058fbe4fb3.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/obj/003f38058fbe4fb3.txt  
replace data/obj/0079e185d9959eba.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data/obj/0079e185d9959eba.jpg  
  inflating: data/obj/0079e185d9959eba.txt  
  inflating: data/obj/00dbe48c211c9e71.jpg  
  inflating: data/obj/00dbe48c211c9e71.txt  
  inflating: data/obj/00f08ae9f7759951.jpg  
  inflating: data/obj/00f08ae9f7759951.txt  
  inflating: data/obj/01cf0d5b15633f1b.jpg  
  inflating: data/obj/01cf0d5b15633f1b.txt  
  inflating: data/obj/01e4eb8072609eef.jpg  
  inflating: data/obj/01e4eb8072609eef.txt  
  inflating: data/obj/020bf2f46d778048.jpg  
  inflating: data/obj/020bf2f46d778048.txt  
  inflating: data/obj/02577a2e2e0ca961.jpg  
  inflating: data/obj/026bcdedce1a3815.jpg  
  infl

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.708008 0.663250 0.458984 0.673499

['15', '0.708008', '0.663250', '0.458984', '0.673499']
0 data/obj/d4ed7b91017de7bf.txt
0 0.708008 0.663250 0.458984 0.673499
15 0.457520 0.581699 0.387695 0.360784

['15', '0.457520', '0.581699', '0.387695', '0.360784']
1 data/obj/a222634f40eb623b.txt
0 0.457520 0.581699 0.387695 0.360784
15 0.406250 0.696925 0.339844 0.606149

['15', '0.406250', '0.696925', '0.339844', '0.606149']
2 data/obj/45060e763e76a2e9.txt
0 0.406250 0.696925 0.339844 0.606149
15 0.445801 0.693436 0.446289 0.557263

['15', '0.445801', '0.693436', '0.446289', '0.557263']
3 data/obj/954baaf90a6905f9.txt
0 0.445801 0.693436 0.446289 0.557263
15 0.488281 0.246528 0.128906 0.256944

['15', '0.488281', '0.246528', '0.128906', '0.256944']
4 data/obj/5c4ac1d3097d9bad.txt
0 0.488281 0.246528 0.128906 0.256944
15 0.487630 0.625000 0.641927 0.736328

['15', '0.487630', '0.625000', '0.641927', '0.736328']
5 data/obj/0e5426035c36bb4f.txt
0 0.487630 0.625000 0.641927 0.736328
15 0.47607

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/c17d7f79264386a5.jpg', 'data/obj/84f0d2ab21918b21.jpg', 'data/obj/f074d5978af8dcef.jpg', 'data/obj/1122cc4d35422190.jpg', 'data/obj/831dd8298db3a1d4.jpg', 'data/obj/26c1751789680627.jpg', 'data/obj/292cbcbedef826ed.jpg', 'data/obj/5e371acb01c9fb82.jpg', 'data/obj/ab44f65b943be4ab.jpg', 'data/obj/eb674c106748f203.jpg', 'data/obj/04b81afcb832ecc1.jpg', 'data/obj/e707f013e53421b2.jpg', 'data/obj/c3e2486456a6fd61.jpg', 'data/obj/33472c3ef8762fe8.jpg', 'data/obj/6106c1eb5ce75756.jpg', 'data/obj/e3ce17ebf17ca9fc.jpg', 'data/obj/2bad4329bf5ed845.jpg', 'data/obj/d4ed7b91017de7bf.jpg', 'data/obj/03d68359159b4d0b.jpg', 'data/obj/c5e4feaa457245b8.jpg', 'data/obj/7796ec8a47553e96.jpg', 'data/obj/87466bb270c7c97c.jpg', 'data/obj/911536da8554817d.jpg', 'data/obj/9c0760e814d2cbc0.jpg', 'data/obj/2e13d00fb932574a.jpg', 'data/obj/0b11c3efc43e5094.jpg', 'data/obj/850e97f8eef9bfc1.jpg', 'data/obj/24cc82665712bb80.jpg', 'data/obj/e51c91ec121cddf2.jpg', 'data/obj/1a4e8ed2f16544e0.jpg', 'data/obj

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 95899, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.792628), count: 3, class_loss = 0.323472, iou_loss = 0.166613, total_loss = 0.490085 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.537862), count: 1, class_loss = 0.041781, iou_loss = 0.218884, total_loss = 0.260665 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 95903, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.793860), count: 4, class_loss = 0.424821, iou_loss = 0.161349, total_loss = 0.586170 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000010, iou_loss = 0.000000, total_loss = 0.000010 
v3 (mse 